This Notebook will take a Youtube Playlist and convert all videos to MP3:s , which will be stored on a folder on your Google Drive.

In [ ]:
#Initialize
import os
def my_mkdirs(folder):
  if os.path.exists(folder)==False:
    os.makedirs(folder)
my_mkdirs('/content/tmp/')

In [2]:
#Reinstall youtube_dl because the version on Colab is outdated
%pip -qqq install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
%pip -qqq install youtube_transcript_api datasets webvtt-py srt pydub


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
import yt_dlp
import re
import os


In [5]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
def get_video_id(youtube_url):
    video_id = re.findall(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', youtube_url)
    return video_id[0] if video_id else None



In [36]:
def extract_audio(youtube_url, output_path='ainiacs-project/tts/dataset'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'writeautomaticsub': True,
        'writesubtitles': True,
        'subtitleslangs': ['ar'],
        'ffmpeg_location': '/usr/local/bin/ffmpeg',
        # 'subtitlesformat': 'srt'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(youtube_url, download=True)
            return info['title']
        except Exception as e:
            print(f"Error extracting audio: {str(e)}")
            return None

In [24]:
import pandas as pd

from datasets import Dataset

In [38]:
df = pd.read_csv('data_sources.csv')
df.head(20)

,Channel,Link,Video,Transcripts' type (auto/manual),Content Type,Unnamed: 5,Unnamed: 6,"Instructions: Channels should contain as much darija as possible (few codeswitching / classical arabic..), and contents (to have difference vocabulary)"
0,Fakhr-din Stories,https://youtube.com/playlist?list=PLy-ucrR8Z-F...,https://youtube.com/playlist?list=PLy-ucrR8Z-F...,auto,NaN,NaN,NaN,NaN
1,Najib El Mokhtari,https://youtu.be/3a6qHvFLNOY,https://youtu.be/3a6qHvFLNOY,auto,NaN,NaN,NaN,NaN
2,Arabi21 - عربي21,https://youtu.be/ebsEumoRao0,https://youtu.be/ebsEumoRao0,auto,NaN,NaN,NaN,NaN
3,Arabi21 - عربي22,https://www.youtube.com/watch?v=i2zS9q5RB3A,https://www.youtube.com/watch?v=i2zS9q5RB3A,auto,NaN,NaN,NaN,NaN
4,Dr Oussama Benchekroun - المغرب,https://youtu.be/qyJKbKLNXzg,https://youtu.be/qyJKbKLNXzg,auto,NaN,NaN,NaN,NaN
5,Kifache tv | كيفاش تيفي,https://youtu.be/9wX_znEMBKQ?list=PLDaUgVEitTO...,https://youtu.be/9wX_znEMBKQ?list=PLDaUgVEitTO...,auto,NaN,NaN,NaN,NaN
6,Simo Sedraty,https://youtu.be/iExlaNNXdxk,https://youtu.be/iExlaNNXdxk,auto,NaN,NaN,NaN,NaN
7,Simo Sedraty,https://youtu.be/DhQoPUcarfg?si=3Ry_oJbWrvCRSTre,https://youtu.be/DhQoPUcarfg?si=3Ry_oJbWrvCRSTre,auto,NaN,NaN,NaN,NaN
8,Fakhr-din L'hamri,https://youtu.be/WSRRzDtEWiM,https://youtu.be/WSRRzDtEWiM,auto,NaN,NaN,NaN,NaN
9,ksiyer,https://youtu.be/tNk4NHl4iFY,https://youtu.be/tNk4NHl4iFY,auto,NaN,NaN,NaN,NaN


In [9]:
def get_transcript(video_id, language='ar'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        return transcript
    except Exception as e:
        print(f"Error fetching transcript: {str(e)}")
        try:
            transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
            available_transcripts = transcript_list.find_transcript([language])
            if available_transcripts:
                return available_transcripts.fetch()
            else:
                print("No suitable transcript found.")
                return None
        except Exception as e:
            print(f"Error fetching available transcripts: {str(e)}")
            return None


In [39]:
def pipeline(youtube_url):

  audio_title = extract_audio(youtube_url)

  return audio_title


In [ ]:
def format_and_save_transcript(transcript: list, output_file: str):
  data = pd.DataFrame(columns=['start', 'duration', 'text'])
  for entry in transcript:
    data.loc[len(data)] = [f"{entry['start']:.2f}", f"{entry['duration']:.2f}", entry['text']]
  data.to_csv(output_file, index=False)

In [ ]:
format_and_save_transcript(transcript)

In [40]:
df.shape[0]

17

In [ ]:
for i in range(df.shape[0]):
    audio_title = pipeline(df['Video'][i])


[youtube:tab] Extracting URL: https://youtube.com/playlist?list=PLy-ucrR8Z-FW0zQGdK5ZKq1QY7CY37kHn&si=QwSjynqJMwaMwGb9
[youtube:tab] PLy-ucrR8Z-FW0zQGdK5ZKq1QY7CY37kHn: Downloading webpage


[youtube:tab] PLy-ucrR8Z-FW0zQGdK5ZKq1QY7CY37kHn: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Stories
[youtube:tab] PLy-ucrR8Z-FW0zQGdK5ZKq1QY7CY37kHn page 1: Downloading API JSON


[youtube:tab] PLy-ucrR8Z-FW0zQGdK5ZKq1QY7CY37kHn page 1: Downloading API JSON


[youtube:tab] PLy-ucrR8Z-FW0zQGdK5ZKq1QY7CY37kHn page 1: Downloading API JSON


[youtube:tab] PLy-ucrR8Z-FW0zQGdK5ZKq1QY7CY37kHn page 1: Downloading API JSON


[youtube:tab] Playlist Stories: Downloading 60 items of 60
[download] Downloading item 1 of 60
[youtube] Extracting URL: https://www.youtube.com/watch?v=EKAsJDq0Lzk
[youtube] EKAsJDq0Lzk: Downloading webpage
[youtube] EKAsJDq0Lzk: Downloading tv client config
[youtube] EKAsJDq0Lzk: Downloading player 1080ef44
[youtube] EKAsJDq0Lzk: Downloading tv player API JSON
[youtube] EKAsJDq0Lzk: Downloading ios player API JSON
[youtube] EKAsJDq0Lzk: Downloading m3u8 information
[info] EKAsJDq0Lzk: Downloading subtitles: ar
[info] EKAsJDq0Lzk: Downloading 1 format(s): 251
Deleting existing file ainiacs-project/tts/dataset/سرق 232  مليار بطريقة عبقرية!.ar.vtt
[info] Writing video subtitles to: ainiacs-project/tts/dataset/سرق 232  مليار بطريقة عبقرية!.ar.vtt
[download] Destination: ainiacs-project/tts/dataset/سرق 232  مليار بطريقة عبقرية!.ar.vtt
[download] 100% of  126.43KiB in 00:00:02 at 47.76KiB/s
[download] ainiacs-project/tts/dataset/سرق 232  مليار بطريقة عبقرية!.webm has already been downloade

In [ ]:
import webvtt

def vtt_to_srt(vtt_file):
    vtt = webvtt.read(vtt_file)
    vtt.save_as_srt(vtt_file.replace('.vtt', '.srt'))

In [ ]:
vtt_to_srt('/content/ainiacs-project/tts/dataset/أجي تفهم تحرير سعر صرف الدرهم  تعويم الدرهم.ar.vtt')

In [ ]:
#loop over all files in the folder
for file in os.listdir('/content/ainiacs-project/tts/dataset'):
    if file.endswith('.vtt'):
        vtt_to_srt(os.path.join('/content/ainiacs-project/tts/dataset', file))

In [6]:
!pip install -qqq srt pydub

In [50]:
import srt, io
from pydub import AudioSegment
from datasets import Dataset, Features, Audio, Value
from tqdm.notebook import tqdm


# Function to parse the SRT file
def parse_srt(srt_file):
    with open(srt_file, 'r') as f:
        subtitles = srt.parse(f.read())
    return subtitles

# Function to split audio based on SRT timings
def split_audio(audio_file, srt_file, output_dir, dataset_path, index_file):
    # Load audio file
    audio = AudioSegment.from_file(audio_file)

    # Parse the SRT file
    subtitles = parse_srt(srt_file)

    data = {
        "audio": [],
        "text": []
    }

    # Split audio based on subtitle timings
    for idx, subtitle in tqdm(enumerate(subtitles)):
        start_time = subtitle.start.total_seconds() * 1000  # Convert to milliseconds
        end_time = subtitle.end.total_seconds() * 1000    # Convert to milliseconds

        # Extract the corresponding chunk from the audio
        chunk = audio[start_time:end_time]

        audio_buffer = io.BytesIO()
        chunk.export(audio_buffer, format="mp3")
        audio_buffer.seek(0)

        # Save chunk to a new file
        chunk.export(f"{output_dir}/chunk_{index_file}_{idx + 1}.mp3", format="mp3")
        # print(f"Saved chunk_{idx + 1}.mp3")

        data["audio"].append({"bytes": audio_buffer.read()})
        data["text"].append(subtitle.content)


    features = Features({
        "audio": Audio(sampling_rate=22050),  # Adjust sampling rate as needed
        "text": Value("string")
    })

    # Create HF Dataset
    dataset = Dataset.from_dict(data, features=features)

    # Save the dataset locally
    # dataset.save_to_disk(dataset_path)
    # print(f"Dataset saved to {dataset_path}")
    return dataset

In [51]:
def offset_text_column(dataset):
    """
    Offsets the 'text' column in a Hugging Face Dataset by one row and removes rows with even indices.

    Args:
        dataset (datasets.Dataset): The input dataset with 'audio' and 'text' columns.

    Returns:
        datasets.Dataset: A new dataset with the 'text' column offset by one row and even-index rows removed.
    """
    # Ensure the dataset has the required columns
    if 'audio' not in dataset.column_names or 'text' not in dataset.column_names:
        raise ValueError("The dataset must have 'audio' and 'text' columns.")

    # Offset the 'text' column by one row
    text_column = dataset['text']
    offset_text_column = text_column[1:] + [None]  # Shift text column by one row, filling last row with None

    # Create a new dataset with the updated 'text' column
    dataset_with_offset = dataset.map(
        lambda example, idx: {'text': offset_text_column[idx]},
        with_indices=True
    )

    # Remove rows with even indices
    filtered_dataset = dataset_with_offset.filter(lambda example, idx: idx % 2 != 0, with_indices=True)

    return filtered_dataset

In [ ]:
!rm -rf tmp
!mkdir -p tmp/dataset
# Example usage
audio_file = '/content/dataset/آجي نطلقوها تسرح على جديد مدونة الأسرة.mp3'
srt_file = '/content/dataset/آجي نطلقوها تسرح على جديد مدونة الأسرة.ar.srt'
output_dir = 'tmp'  # Specify the output directory for chunks
dataset_path = 'tmp/dataset'  # Specify the path to save the dataset

ds = split_audio(audio_file, srt_file, output_dir, dataset_path)
offset_ds = offset_text_column(ds)
offset_ds.push_to_hub('test_stt_audio_offsetted_nayf_aguerd')


0it [00:00, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/2304 [00:00<?, ? examples/s]

Dataset saved to tmp/dataset


Map:   0%|          | 0/2304 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2304 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/medmac01/test_stt_audio_offsetted_nayf_aguerd/commit/3a66bca2b393262f372e09524e5fbb9c15cb5437', commit_message='Upload dataset', commit_description='', oid='3a66bca2b393262f372e09524e5fbb9c15cb5437', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/medmac01/test_stt_audio_offsetted_nayf_aguerd', endpoint='https://huggingface.co', repo_type='dataset', repo_id='medmac01/test_stt_audio_offsetted_nayf_aguerd'), pr_revision=None, pr_num=None)

In [55]:
from datasets import concatenate_datasets, DatasetDict

def process_and_push_dataset(folder_path: str, hf_repo_id: str):
    """
    Processes all audio files in the given folder,
    concatenates the resulting datasets, and pushes to HF Hub.

    Args:
        folder_path (str): Path to the folder containing .mp3 and .srt files.
        hf_repo_id (str): The Hugging Face repository ID to push the dataset.
    """
    datasets_list = []

    # Iterate over all files in the folder
    # print(os.listdir(folder_path)[0].endswith(".mp3"))

    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder {folder_path} does not exist.")
        return

    # List all files in the directory
    files = os.listdir(folder_path)
    print(f"Found {len(files)} files in {folder_path}")

    count = 0
    # Iterate over all files in the folder
    try:
        for file in files:
            if file.endswith(".mp3"):
                audio_path = os.path.join(folder_path, file)
                srt_path = os.path.splitext(audio_path)[0] + ".ar.srt"

                if os.path.exists(srt_path):
                    print(f"Found matching subtitle for {file}: {srt_path}")
                    ds = split_audio(audio_path, srt_path, "tmp", "tmp/dataset", count)
                    dataset = offset_text_column(ds)
                    datasets_list.append(dataset)
                    count += 1
                else:
                    print(f"Warning: No subtitle found for {file}")
    except Exception as e:
        print(f"Error processing file, uploading already done ds: {str(e)}")
        final_dataset = concatenate_datasets(datasets_list)

        # Convert to DatasetDict (optional, useful for structured datasets)
        dataset_dict = DatasetDict({"train": final_dataset})

        # Push to Hugging Face Hub
        dataset_dict.push_to_hub(hf_repo_id)
        print(f"Dataset pushed to {hf_repo_id}")

    # for file in os.listdir(folder_path):
    #     if file.endswith(".mp3"):
    #         audio_path = os.path.join(folder_path, file)
    #         srt_path = os.path.splitext(audio_path)[0] + ".srt"

    #         if os.path.exists(srt_path):
    #             ds = split_audio(audio_path, srt_path, "tmp", "tmp/dataset")
    #             dataset = offset_text_column(ds)
    #             datasets_list.append(dataset)

    # Ensure there's at least one dataset to concatenate
    if datasets_list:
        final_dataset = concatenate_datasets(datasets_list)

        # Convert to DatasetDict (optional, useful for structured datasets)
        dataset_dict = DatasetDict({"train": final_dataset})

        # Push to Hugging Face Hub
        dataset_dict.push_to_hub(hf_repo_id)
        print(f"Dataset pushed to {hf_repo_id}")

    else:
        print("No valid datasets found!")

In [54]:
datasets_list

NameError: name 'datasets_list' is not defined

In [ ]:
process_and_push_dataset("/content/dataset", "full_tts_dataset")

Found 54 files in /content/dataset
Found matching subtitle for قصة الناس ： لم أرد العيش في الحرام (حلقة كاملة).mp3: /content/dataset/قصة الناس ： لم أرد العيش في الحرام (حلقة كاملة).ar.srt


0it [00:00, ?it/s]

## TODO : jm3 kolshi emporter (single func: from csv to final ds)

add some preprocessing (remove special keywords [music] [clatter]...)

add checkpointing

In [12]:
# copy the dataset from drive to working dir
!cp -r /content/drive/MyDrive/ainiacs-project/tts/dataset .

In [ ]:
!cp -r tmp /content/drive/MyDrive/

In [ ]:
!python3 -m pip install -U https://github.com/bindestriche/srt_fix/archive/refs/heads/master.zip

     \ 9.2 MB 23.0 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for srt_fix: filename=srt_fix-2023.1.2-py3-none-any.whl size=5315 sha256=d14d7b4dd7b32dedbb9f8442eedffb46dc8a27d801a9d95bf45b5ea0bf4f5646
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ux2p_8p/wheels/eb/08/35/aba28d3b3d54b3f6c7435ad00e5c8ba931285c68f513f6396b
Successfully built srt_fix


In [ ]:
from datasets import Dataset

In [ ]:
ds = Dataset.load_from_disk('tmp/dataset')

In [ ]:
new_ds = offset_text_column(ds)

Map:   0%|          | 0/286 [00:00<?, ? examples/s]

Filter:   0%|          | 0/286 [00:00<?, ? examples/s]

In [ ]:
new_ds.push_to_hub('test_audio_tts_offset_rmv')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/143 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/medmac01/test_audio_tts_offset_rmv/commit/8b627c0ca7bce7426d1a237bf009cc7ffd0608f6', commit_message='Upload dataset', commit_description='', oid='8b627c0ca7bce7426d1a237bf009cc7ffd0608f6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/medmac01/test_audio_tts_offset_rmv', endpoint='https://huggingface.co', repo_type='dataset', repo_id='medmac01/test_audio_tts_offset_rmv'), pr_revision=None, pr_num=None)

In [ ]:
new_ds

Dataset({
    features: ['audio', 'text'],
    num_rows: 143
})

In [ ]:
ds.push_to_hub('test_audio_tts_2')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/286 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/medmac01/test_audio_tts_2/commit/3c57ad6bce061e6cffec338ee3bb7e60be9ef681', commit_message='Upload dataset', commit_description='', oid='3c57ad6bce061e6cffec338ee3bb7e60be9ef681', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/medmac01/test_audio_tts_2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='medmac01/test_audio_tts_2'), pr_revision=None, pr_num=None)

In [ ]:
type(transcript)

list

In [10]:
import os
import json
from pydub import AudioSegment
from datasets import Dataset

def split_audio(audio_file, transcript, output_dir, dataset_name):
    """
    Splits an audio file into chunks based on the JSON metadata file
    and saves the chunks while creating a Hugging Face dataset.

    Parameters:
    - audio_file: Path to the input audio file.
    - json_file: Path to the JSON file with `text`, `start`, and `duration`.
    - output_dir: Directory to save audio chunks.
    - dataset_name: Name of the Hugging Face dataset.

    Returns:
    - Hugging Face dataset with audio data and text.
    """
    # Load the audio file
    audio = AudioSegment.from_file(audio_file)

    data = transcript

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Initialize dataset records
    dataset_records = []

    # Process each entry in the JSON file
    for idx, entry in enumerate(data):
        start_ms = int(entry['start'] * 1000)
        end_ms = int(entry['duration'] * 1000)
        text = entry['text']

        # Extract the audio chunk
        audio_chunk = audio[start_ms:end_ms]

        audio_buffer = io.BytesIO()
        audio_chunk.export(audio_buffer, format="mp3")
        audio_buffer.seek(0)

        # Save the chunk to a temporary file
        chunk_path = os.path.join(output_dir, f"chunk_{idx + 1}.mp3")
        audio_chunk.export(chunk_path, format="mp3")

        # Append to the dataset record
        dataset_records.append({
            "audio": {"bytes": audio_buffer.read()},  # Store raw audio data
            "text": text,
            "start": entry['start'],
            "duration": entry['duration']
        })

    features = Features({
        "audio": Audio(sampling_rate=22050),  # Adjust sampling rate as needed
        "text": Value("string"),
        "start": Value("string"),
        "duration": Value("string")
    })

    # Create a Hugging Face dataset
    hf_dataset = Dataset.from_dict({
        "audio": [record["audio"] for record in dataset_records],
        "text": [record["text"] for record in dataset_records],
        "start": [record["start"] for record in dataset_records],
        "duration": [record["duration"] for record in dataset_records]
    }, features=features)

    # Save the dataset locally
    hf_dataset.save_to_disk(dataset_name)

    return hf_dataset


In [ ]:
audio_file = '/content/dataset/العلاج من الإكتئاب بدون أدوية - ما تقوله الأبحات العلمية الحديتة.mp3'

output_dir = "audio_chunks"
dataset_name = "hf_dataset"

# Call the function
dataset = split_audio(audio_file, transcript, output_dir, dataset_name)
print(f"Hugging Face Dataset saved to: {dataset_name}")


Saving the dataset (0/1 shards):   0%|          | 0/174 [00:00<?, ? examples/s]

Hugging Face Dataset saved to: hf_dataset


In [ ]:
dataset.push_to_hub("test_audio_tts_3")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/415 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/medmac01/test_audio_tts_3/commit/59be81926c336708685770d9163063c013a395d8', commit_message='Upload dataset', commit_description='', oid='59be81926c336708685770d9163063c013a395d8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/medmac01/test_audio_tts_3', endpoint='https://huggingface.co', repo_type='dataset', repo_id='medmac01/test_audio_tts_3'), pr_revision=None, pr_num=None)